In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import networkx as nx
import os
import sys
sys.path.append('..')
from py_files.michele_algo import *

# Import Data

In [ ]:
def inspect_network(folder):
    print(folder)
    creation_date = str(folder.split('/')[-3])
    df = pd.DataFrame(columns = ['name', 'scrape_date', '#nodes', '#edges', 'average_shortest_path_length', 'Generalized_euclidean', 'Our_polarity_score'])
    for file in os.listdir(folder):
        graph = nx.read_gexf(folder+file)
        graph = nx.to_undirected(graph)
        S = [graph.subgraph(c).copy() for c in sorted(nx.connected_components(graph), key=len, reverse=True)]
        graph = S[0]
        name = file[:-5]

        attr_dict = {
            'name' : name,
            'scrape_date' : creation_date, 
            '#nodes' : len(graph.nodes()),
            '#edges' : len(graph.edges()),
            'average_shortest_path_length' : nx.average_shortest_path_length(graph),
            'Generalized_euclidean' : do_ge(graph),
            'Our_polarity_score' : driver_g(graph) 
        }
        df = df.append(attr_dict, ignore_index = True)
    return df

In [ ]:
main_dir = '../data/date_folders/'
First = True
for folder in os.listdir(main_dir):
    print(main_dir+folder+'/graphs/')
    if First:
        df = inspect_network(main_dir+folder+'/graphs/')
        First = False
    else:
        try:
            df = inspect_network(main_dir+folder+'/graphs/').append(df)
        except:
            pass

In [ ]:
#df = df.set_index('name')

In [ ]:
df

In [ ]:
sns.pairplot(df, 
             #hue='name'
             )

# Barplots of main measures

In [ ]:
# generalized euclidean (michele measure 1)
df_GE = df.copy().sort_values('Generalized_euclidean')
bar_GE = sns.barplot(y=df_GE['name'], x=df_GE['Generalized_euclidean'])

In [ ]:
# Correlation between ideological distance and comment sentiment
df_correlation = df.copy().sort_values('Our_polarity_score')
bar_correlation = sns.barplot(y=df_correlation['name'], x=df_correlation['Our_polarity_score'])

# Calculate stats per Network

# Check Correlations

# lineplots?